In [13]:
%matplotlib inline
import io
import json
import multiprocessing
import os
import time
import traceback
from functools import partial

import numpy as np
import requests
from azure.cognitiveservices.search.imagesearch import ImageSearchAPI
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
from skimage import img_as_ubyte
from skimage.io import imread, imsave
from skimage.transform import resize
from tqdm import tqdm_notebook as tqdm

In [2]:
search_key = ''
cv_key = ''
cv_endpoint = 'https://sm-cv.cognitiveservices.azure.com/'

search_client = ImageSearchAPI(CognitiveServicesCredentials(search_key))
cv_client = ComputerVisionClient(cv_endpoint, CognitiveServicesCredentials(cv_key))

search_terms = ["tiger sleeping",
                "tiger eating",
                "tiger walking",
                "tiger resting",
                "tiger running",
                "tiger grooming",
                "tiger swimming"]
desired_properties = ['tiger', 'mammal', 'animal', 'cat', 'cow']

image_size = 224
crop_margin = 20

In [10]:
n_attempts = 3

def with_retries(action):
    for i in range(n_attempts - 1):
        try:
            return action()
        except Exception as e:
            traceback.print_exc()
            time.sleep(2**i)
    return action()

In [11]:
def process_image(section, image_item):
    image_index, url = image_item
    
    try:
        data = requests.get(url, timeout=1.0).content
        image = imread(io.BytesIO(data))
    except Exception as e:
#         print(e)
        return

    if len(image.shape) == 3 and image.shape[2] == 4:
        image = image[:, :, :3]
    if len(image.shape) == 2:
        image = np.dstack([image] * 3)

    os.makedirs('full_images/{}'.format(section), exist_ok=True)
    filename = 'full_images/{}/{}.jpg'.format(section, image_index)
#     print(filename)
    imsave(filename, img_as_ubyte(image))

    try:
        objects = with_retries(lambda: cv_client.detect_objects(url).objects)
    except Exception as e:
#         traceback.print_exc()
        return
    crop_index = 0
    for obj in objects:
#         print(obj.object_property)
        if obj.object_property not in desired_properties:
            continue

        min_y = max(obj.rectangle.y - crop_margin, 0)
        max_y = obj.rectangle.y + obj.rectangle.h + crop_margin
        min_x = max(obj.rectangle.x - crop_margin, 0)
        max_x = obj.rectangle.x + obj.rectangle.w + crop_margin
        crop = image[min_y:max_y, min_x:max_x]

        crop = resize(crop, (image_size, image_size))

        os.makedirs('crops/{}'.format(section), exist_ok=True)
        filename = 'crops/{}/{}_{}.jpg'.format(section, image_index, crop_index)
#         print(filename)
        imsave(filename, img_as_ubyte(crop))
        crop_index += 1

In [14]:
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count() * 4)
for query in search_terms:
    print('\n> {}\n'.format(query))
    section = query.replace(' ', '_')
    
    images = []
    for i in range(10):
        response = search_client.images.search(query=query,
                                        offset=len(images),
                                        count=150,
                                        image_type="Photo",
                                        min_height=image_size,
                                        min_width=image_size).value
        images += [item.content_url for item in response]
        print("{} -> {} images".format(query, len(images)))
    
    os.makedirs('image_urls', exist_ok=True)
    filename = 'image_urls/{}.json'.format(section)
    print(filename)
    with open(filename, 'w') as f:
        json.dump(images, f)
    
    crop_index = 0
    for item in tqdm(pool.imap_unordered(partial(process_image, section), enumerate(images))):
        pass


> tiger sleeping

tiger sleeping -> 150 images
tiger sleeping -> 300 images
tiger sleeping -> 450 images
tiger sleeping -> 600 images
tiger sleeping -> 750 images
tiger sleeping -> 822 images
tiger sleeping -> 894 images
tiger sleeping -> 966 images
tiger sleeping -> 1038 images
tiger sleeping -> 1110 images
image_urls/tiger_sleeping.json


/anaconda/envs/py36/lib/python3.6/site-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/anaconda/envs/py36/lib/python3.6/site-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/anaconda/envs/py36/lib/python3.6/site-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/anaconda/envs/py36/lib/python3.6/site-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/anaconda/envs/py36/lib/python3.6/site-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/anaconda/envs/py36/lib/python3.6/site-packages/skimage/util

azure.cognitiveservices.vision.computervision.models._models_py3.ComputerVisionErrorException: Input image is too large.
Traceback (most recent call last):
  File "<ipython-input-10-586e950bb6f2>", line 6, in with_retries
    return action()
  File "<ipython-input-11-d084170dce49>", line 22, in <lambda>
    objects = with_retries(lambda: cv_client.detect_objects(url).objects)
  File "/anaconda/envs/py36/lib/python3.6/site-packages/azure/cognitiveservices/vision/computervision/operations/_computer_vision_client_operations.py", line 266, in detect_objects
    raise models.ComputerVisionErrorException(self._deserialize, response)
azure.cognitiveservices.vision.computervision.models._models_py3.ComputerVisionErrorException: Operation returned an invalid status code 'Too Many Requests'
Traceback (most recent call last):
  File "<ipython-input-10-586e950bb6f2>", line 6, in with_retries
    return action()
  File "<ipython-input-11-d084170dce49>", line 22, in <lambda>
    objects = with_retri

azure.cognitiveservices.vision.computervision.models._models_py3.ComputerVisionErrorException: Image URL is not accessible.
Traceback (most recent call last):
  File "<ipython-input-10-586e950bb6f2>", line 6, in with_retries
    return action()
  File "<ipython-input-11-d084170dce49>", line 22, in <lambda>
    objects = with_retries(lambda: cv_client.detect_objects(url).objects)
  File "/anaconda/envs/py36/lib/python3.6/site-packages/azure/cognitiveservices/vision/computervision/operations/_computer_vision_client_operations.py", line 266, in detect_objects
    raise models.ComputerVisionErrorException(self._deserialize, response)
azure.cognitiveservices.vision.computervision.models._models_py3.ComputerVisionErrorException: Image URL is not accessible.
Traceback (most recent call last):
  File "<ipython-input-10-586e950bb6f2>", line 6, in with_retries
    return action()
  File "<ipython-input-11-d084170dce49>", line 22, in <lambda>
    objects = with_retries(lambda: cv_client.detect_ob


> tiger eating

tiger eating -> 150 images
tiger eating -> 300 images
tiger eating -> 450 images
tiger eating -> 600 images
tiger eating -> 730 images
tiger eating -> 732 images
tiger eating -> 862 images
tiger eating -> 992 images
tiger eating -> 1122 images
tiger eating -> 1252 images
image_urls/tiger_eating.json


...

Traceback (most recent call last):
  File "<ipython-input-10-586e950bb6f2>", line 6, in with_retries
    return action()
  File "<ipython-input-11-d084170dce49>", line 22, in <lambda>
    objects = with_retries(lambda: cv_client.detect_objects(url).objects)
  File "/anaconda/envs/py36/lib/python3.6/site-packages/azure/cognitiveservices/vision/computervision/operations/_computer_vision_client_operations.py", line 266, in detect_objects
    raise models.ComputerVisionErrorException(self._deserialize, response)
azure.cognitiveservices.vision.computervision.models._models_py3.ComputerVisionErrorException: Input image is too large.
Traceback (most recent call last):
  File "<ipython-input-10-586e950bb6f2>", line 6, in with_retries
    return action()
  File "<ipython-input-11-d084170dce49>", line 22, in <lambda>
    objects = with_retries(lambda: cv_client.detect_objects(url).objects)
  File "/anaconda/envs/py36/lib/python3.6/site-packages/azure/cognitiveservices/vision/computervision/opera

Traceback (most recent call last):
  File "<ipython-input-10-586e950bb6f2>", line 6, in with_retries
    return action()
  File "<ipython-input-11-d084170dce49>", line 22, in <lambda>
    objects = with_retries(lambda: cv_client.detect_objects(url).objects)
  File "/anaconda/envs/py36/lib/python3.6/site-packages/azure/cognitiveservices/vision/computervision/operations/_computer_vision_client_operations.py", line 266, in detect_objects
    raise models.ComputerVisionErrorException(self._deserialize, response)
azure.cognitiveservices.vision.computervision.models._models_py3.ComputerVisionErrorException: Operation returned an invalid status code 'Too Many Requests'
Traceback (most recent call last):
  File "<ipython-input-10-586e950bb6f2>", line 6, in with_retries
    return action()
  File "<ipython-input-11-d084170dce49>", line 22, in <lambda>
    objects = with_retries(lambda: cv_client.detect_objects(url).objects)
  File "/anaconda/envs/py36/lib/python3.6/site-packages/azure/cognitive


> tiger swimming

tiger swimming -> 149 images
tiger swimming -> 299 images
tiger swimming -> 449 images
tiger swimming -> 599 images
tiger swimming -> 749 images
tiger swimming -> 763 images
tiger swimming -> 776 images
tiger swimming -> 789 images
tiger swimming -> 802 images
tiger swimming -> 815 images
image_urls/tiger_swimming.json


Traceback (most recent call last):
  File "<ipython-input-10-586e950bb6f2>", line 6, in with_retries
    return action()
  File "<ipython-input-11-d084170dce49>", line 22, in <lambda>
    objects = with_retries(lambda: cv_client.detect_objects(url).objects)
  File "/anaconda/envs/py36/lib/python3.6/site-packages/azure/cognitiveservices/vision/computervision/operations/_computer_vision_client_operations.py", line 266, in detect_objects
    raise models.ComputerVisionErrorException(self._deserialize, response)
azure.cognitiveservices.vision.computervision.models._models_py3.ComputerVisionErrorException: Input data is not a valid image.
Traceback (most recent call last):
  File "<ipython-input-10-586e950bb6f2>", line 6, in with_retries
    return action()
  File "<ipython-input-11-d084170dce49>", line 22, in <lambda>
    objects = with_retries(lambda: cv_client.detect_objects(url).objects)
  File "/anaconda/envs/py36/lib/python3.6/site-packages/azure/cognitiveservices/vision/computervisio

Traceback (most recent call last):
  File "<ipython-input-10-586e950bb6f2>", line 6, in with_retries
    return action()
  File "<ipython-input-11-d084170dce49>", line 22, in <lambda>
    objects = with_retries(lambda: cv_client.detect_objects(url).objects)
  File "/anaconda/envs/py36/lib/python3.6/site-packages/azure/cognitiveservices/vision/computervision/operations/_computer_vision_client_operations.py", line 266, in detect_objects
    raise models.ComputerVisionErrorException(self._deserialize, response)
azure.cognitiveservices.vision.computervision.models._models_py3.ComputerVisionErrorException: Input image is too large.
Traceback (most recent call last):
  File "<ipython-input-10-586e950bb6f2>", line 6, in with_retries
    return action()
  File "<ipython-input-11-d084170dce49>", line 22, in <lambda>
    objects = with_retries(lambda: cv_client.detect_objects(url).objects)
  File "/anaconda/envs/py36/lib/python3.6/site-packages/azure/cognitiveservices/vision/computervision/opera